In [2]:
import keras
import numpy as np

In [8]:

path = keras.utils.get_file(
'nietzsche.txt',
origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('coprus lenght:', len(text))

coprus lenght: 600901


In [9]:
# uzycie one hot encoder
maxlen =60 #sekwencje składające sie  z 60 znaków
step =3 #nowa sekwencja jest próbkowana co 3 znaki
sentences = []#zmienna w której zpisywana beda wyodrebione sekwencje
next_char =[]#zmienna w której zpisywana beda kolejne znaki (cele)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_char.append(text[i+maxlen])
print('liczba sekwencji:', len(sentences))


liczba sekwencji: 200281


In [10]:
chars = sorted(list(set(text)))
print('liczba unikatowych znaków:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)#słownik uniktowy znak: index


liczba unikatowych znaków: 59


In [11]:
# dane na któych bedzie trenowany model, twa wektory
print('tworzenie wektorów...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) # wektor o kształcie 
                                                                   # (sequen, maxlen, unique char)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t, char_indices[char]] =1
    y[i, char_indices[next_char[i]]]=1

tworzenie wektorów...


C:\Users\Grzegorz Mróz\AppData\Local\Temp\ipykernel_16748\1301805642.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) # wektor o kształcie
C:\Users\Grzegorz Mróz\AppData\Local\Temp\ipykernel_16748\1301805642.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sen

In [12]:
# generowani danych sekwncyjnych za pomocą LSTM i kasyfikatora DENSE.
from keras import layers

model = keras.models.Sequential()

model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
# użyto one hot encoder daltgo loss=categorical_crossentropy'
model.compile(optimizer=optimizer, loss ='categorical_crossentropy')

In [ ]:
#funjcja która modyfikuje wagi rozkładu prawdopodbiensta, żeby był w zakresie 
#chciwe próbkowanie(najmiejsza ent.) >= wartość wgi >= próbkowanie sochatyczne(losowe, najwieksza entropia)
def sample(preds, temperature=1.0):#temperature paramert modyfikujacy wagi
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas=np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


# PĘTLA WYKONUJE TRENOWANIE MODELU I GENEROWANIE TEKSTU
# jak zmienia sie tekst z udoskanalaniem modelu oraz jak temperature wpływa na strategie próbkowania


import random
import sys

for epoch in range(1,60): #model bedxie trnowany przez 60 epok
    print('epoka:', epoch)
    model.fit(x,y, batch_size=128, epochs=1)#jedna iteracaja trenowania 
    # losowanie tekstu pocżatkowego
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('----Generowanie przy użyciu tekstu początkowego:"' + generated_text +'"')
          
    for temperature in[0.2, 0.5, 1.0, 1.2]:#sprawdzanie wpływu róznych wartości parametru tempaearutre 
        print('----wartość parametru temperature:', temperature)
        sys.stdout.write(generated_text)
          
        for i in range(400):#generowanie 400 znaków
            #wygenerowane znaki zapisane za pomoca one hot
            sampled = np.zeros((1,maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] =1.
              #próbkowanie kolejnego znaku    
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
                  
            generated_text += next_char
            generated_text = generated_text[1:]
                  
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
          

epoka: 1
1565/1565 [==============================] - 138s 86ms/step - loss: 1.9462
----Generowanie przy użyciu tekstu początkowego:"ry vulgarizations, are almost falsifications of the
original"
----wartość parametru temperature: 0.2
ry vulgarizations, are almost falsifications of the
original the stand of the standing and and strends the standing and the man the nothing that in the standing and man the such a man and in the such mensing the strength the standing the consequent the man the standing the standing the and and all the consequent the free the more condection of the spirit the standing the master the form and the strends and the man the consequenter the spirit that is all th
----wartość parametru temperature: 0.5
rends and the man the consequenter the spirit that is all the state the decain the what all the share the simmat there is all perhaps of the world has all in will be man innoble to the condection of more strends and thus and instands and decains of all and under as 

are veold reprived eadi"y all that indeed, it we would have ligitate provim; if detser
of utivign (p
----wartość parametru temperature: 1.2
d, it we would have ligitate provim; if detser
of utivign (presennots
suther
himself such merely experhation which we diswer
may ankillds,"
the parsiogremin has forpting failivency--and powtes--hand scientering to, at the mocratess, bben our accisently outaflest now that everythicks of indmivtless onive allow yelpnest his
domitic music
good as that
the
mem and his judglivenxy, certacnes why
himself from scupies wus preservation surmond; he been pour evilws
epoka: 5
1565/1565 [==============================] - 132s 84ms/step - loss: 1.4542
----Generowanie przy użyciu tekstu początkowego:"t world," when like him, one has the feet of a wind,
the rus"
----wartość parametru temperature: 0.2
t world," when like him, one has the feet of a wind,
the rust the same the strong the success of the same the substance of the spirit the sense of the strongth of th

the most considered in the sentitude of the subject of the subility of the same things and distrush and depting and contradicts of the persing into the subtle to the states and motive of the bothing for and self-supervative the most good, and the subjection of the soul and intellectual interesting of than men, humanity man with the man of the fact the present to the sensely the sunfation for the fine and capacity sense of the command, by a persise that the
----wartość parametru temperature: 1.0
ine and capacity sense of the command, by a persise that the results is in us, through the opposed skelfly very to mrine!--nothing man suse we plaise. a comment believesed geption of solitud and reads, wherebow, in every owizes stranger, at and penilines he succealifieation and viturcely for, or
and expresent for they bentement and thus at repavation all heroper ffreftitid place. a
understand the forgutigity are late and cashers, the give religion, to mora
----wartość parametru temperature: 1.2


falsehood and nonsense from sheer exhaustions of the superficiality of the spirit the superficialities of the spirit that it is a man is a problem of the same and as a such a sentiment of the spirit and such a proposition of the feeling in the subject it is the present desire in the fundamental instinct of the proposition of the spirit and also in the same and in the spirit the subject is not be a problem of the subjective and considerati
----wartość parametru temperature: 0.5
ubject is not be a problem of the subjective and consideration and instinct and single the sentiment and call as degree to him a really of the superfuling as a attere with morality of the put it is a superficiality of the concedient and sublime spirit in the individual and consideration of philosopher, and it will to have wears to the fearing the experiences, the consequently of the sloly of the fact
at least must be those through truth and every do the re
----wartość parametru temperature: 1.0
act
at least must 

is into the aoblomood, it ma trame thinks, the . compeykien y'ahumments
autholous exception, as hus and labority the strengt on the
idarually tearl youren, and have thature himself--for, however.--is: one concerned by the "us and fundamentraegreas--very deta with the uself smey fraderaining even focrow, expense
nouss tmaie sympathy, as during si. to fen is a ve,itive a
epoka: 16
1565/1565 [==============================] - 283s 181ms/step - loss: 1.3434
----Generowanie przy użyciu tekstu początkowego:"also of the loathsome and dreadful, still greater discoverer"
----wartość parametru temperature: 0.2
also of the loathsome and dreadful, still greater discoverer of the same and such a servical soul, and the servical serious of the same and the suffer the subjection of the same as a presenting and the same and such a stronger and the servical serious of the same and the same and the same and such a soul the superficians of the subject and and such a some perhaps and the same and some part

 fortunately experience. the fear of the play, the senses, that us opinion that his new naties that =accordance would have highres. the ideas, which one is comingthonce that perhaps "much worlded or only spectits, of too highest
hypothesis etiches referrand all flight of farst dain as meanbary to
know
all with les avesely in an amd, prewer cuntion soot or winds become accordance, even it our delused seriously
the pleasoses and
"power also made all as ease 
----wartość parametru temperature: 1.2
ed seriously
the pleasoses and
"power also made all as ease woman;"-external fecres which has ! play
with le it io of the predetenned patided ent.t--if peace,
different certain propures to nay and cafa-impired himself. almosphed to made gnew. to a matter would do
ofweral just in europes to
be delusion it oney astander ueffect,
hard sullpping, falsess: heves bound and pwectoned whither exjeration
arist what.


8

the ignoary, were know we atures not he goin
epoka: 20
1565/1565 [==================

perspectives concerning the future, calculated and say the sense and states of the same an accustom of the spirit of the same things a soul-the self-and something the spirit of the spirit the same to the spirit to the same an allow the same an at the same an an action of the faith of the same any the companismonce of the same an accustom of the same an accustomed the spirit of the same an an interestion of the spirit of the same an in th
----wartość parametru temperature: 0.5
he same an an interestion of the spirit of the same an in the disappors and a day from the delicate to men in the same time in the
young state of
the strongest themselves by the consequence of morals, which the strongest need of the same animal at what is an organical sense of
the complex-an for the sense all the facts of history of the most consequence, the consequence and commen all, but what allow in such offer of the feeling present be historical self 
----wartość parametru temperature: 1.0
low in such offer o

hos absolute
threas tpe enminioue deptificle to "the a malacizy-farrou whate-demaities me deep of europe is, apptis truel: through victor
our
prevary free ssy
things, to daritation would circulus," something towards race--"people, man
! in ips us will
been ximming"; evidence with respecteps its us.., any one; thy elenaition, harmaring and only
epoka: 27
1565/1565 [==============================] - 711s 454ms/step - loss: 1.3052
----Generowanie przy użyciu tekstu początkowego:"g to absolute ethics; but
after each new ethical adjustment,"
----wartość parametru temperature: 0.2
g to absolute ethics; but
after each new ethical adjustment, the same as a sense of the saint and sense of the same time the same as a sense of the same as a most sense of the same as the sense of the same as a sense of the same as the saint with the same as a modes of the same as the same as a moral and superficial sense of the same as a sense of the same as the same as a sense of the same as a sense of the same s

d of the accied, different state into concenting the very with addition a side many his inhumance of which a time christian thus is them words of
life, with himself; so recognized
or
fattevemperence existoring and pabited to bain
feal innocuars us
lod in hereas, be overholops ourselved what reckons of shorts of and sunitys are uson of
recip always our desepude
within one has been fears
areave.--thiss bear qualition.=--every last bad self-contemplative so i
----wartość parametru temperature: 1.2
iss bear qualition.=--every last bad self-contemplative so inall cutting
ethicring
graxifiates: is an additance and
neeknersement) and arpar
comparagaly.o
of reado-tautions.erent
thus plind for no loveion: outhon which evil mishict attenur? -how it feltdly through hones has
germens
him while antie
sagrans
who no transfbpiment to referriety of the usse of the eavisto: sum of
mentakion of the latter of love looks natural world," aneolown. "for betiniof imiti
epoka: 31
1565/1565 [==================

the same time the spirit to the spirit and specialize as a person which the world of the spirit of the spirit the proper which the conscience of the world of the same as a period of the faith of the spirit as it is also world of the profound and such as the same as a sense of the same and all the spirit of the same and and perhaps as it is the assumption of the same as a senses of the faith of the 
----wartość parametru temperature: 0.5
 the assumption of the same as a senses of the faith of the child lefted and hence
and masters, and in the character something for
in the end as the world in the form of all so calling is as the faith of all the plenture of the proper the
spirit of the senteration of the world of the fact in the soul, it we do not desires, so into the more extends
to any one who has not the false outsest of the factered also seem it with them of the acts of the had to the 
----wartość parametru temperature: 1.0
ctered also seem it with them of the acts of the had to the